# SOI1010 Machine Learning II - Assignment #3  
2022094093 Kim Dohoon

Due: December 8, 2023 11:59 pm

You know the drill. The submission should include a code (both link to the colab and .py format) and a report that has answers to the questions and results. Use PyTorch (or TensorFlow/JAX). Also, minimize the use of numpy. There will be mark deductions if numpy is used when PyTorch is preferable or should be used. Marks will be deducted if the submission does not include the requested files. DO NOT use other libraries, such as scikit-learn/sklearn, to use a model you are supposed to implement. Using sklearn or any other third library or already built-in functions that you are asked to implement will result in 0 mark. Also, if an assignment asks you to implement some model, that means you shouldn’t use the built-in implementation from any library for that model in the first place.

NEW guidelines:
1. Do NOT copy codes from references/online resources, especially from DACON/Kaggle. If you get caught copying/combining parts of the codes that are being shared online, you will get 0.
2. Do not use test data (from test.csv) for training your model. Test data should only be used for test. If the results are not reproducible, you will get 0.
3. Do not use external data. Stick with what you are provided with from Kaggle compe- tition.
4. Do not use a pre-trained model. You should train a model from scratch.
5. In other words, use only train data (from train.csv) for training your model.

### Kaggle competition: Sign Language Recognition  


a) The competition/dataset can be found [here](https://www.kaggle.com/competitions/korean-sign-language-recognition-soi1010-2023/overview).

In [171]:
!kaggle competitions download -c korean-sign-language-recognition-soi1010-2023

korean-sign-language-recognition-soi1010-2023.zip: Skipping, found more recently modified local copy (use --force to force download)


b) Your task is to achieve as high performance as possible (without cheating, of course). You can use any model you want. But, I suggest that you start with MLP and Convolutional Neural Networks (CNN). I strongly encourage you to research and study advanced neural network models and implement them to get better performance.

In [1]:
# Utils
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from PIL import Image
from tqdm import tqdm

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader
from torch.autograd import Variable
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image

# etc
from CustomDataLoader import CustomDataset
from model_util import train_loop, evaluate_loop, test_predict, train_eval

import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kdhyu. Use `wandb login --relogin` to force relogin


True

In [2]:
# Device Setting
#device = (
#    'cuda'
#    if torch.cuda.is_available()
#    else 'mps'
#    if torch.backends.mps.is_available()
#    else 'cpu'
#)
device = 'cpu'
print(f"Now using {device} device")

Now using cpu device


In [3]:
class MoLU(nn.Module):
    """Moderate Adaptive Linear Units, MoLU
    https://arxiv.org/abs/2302.13696"""
    def __init__(self, a=2, b=2):
        super(MoLU, self).__init__()
        self.a = a
        self.b = b

    def forward(self, x):
        x = x * torch.tanh(self.a * torch.exp(self.b * x))
        return x

In [4]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=11):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 64, 11, 4, 2),
            nn.ReLU(),
            nn.LocalResponseNorm(5),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 192, 5, 1, 2),
            nn.ReLU(),
            nn.LocalResponseNorm(5),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(192, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.fc = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [28]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 8, 2),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(8, 16, 2, 1, 1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, 2, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 2, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.fc = nn.Sequential(
            nn.Linear(576, 288),
            nn.ReLU(),
            nn.Linear(288, 11)
        )
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x = self.conv(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [13]:
class CNN2(nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 8, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(8, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(1600, 200),
            nn.ReLU()
        )
        self.fc2 = nn.Linear(200, 11)
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x1 = self.conv(x)
        x2 = self.conv(torch.rot90(x, k=1, dims=[2, 3]))
        x3 = self.conv(torch.rot90(x, k=2, dims=[2, 3]))
        x4 = self.conv(torch.rot90(x, k=3, dims=[2, 3]))
        x = torch.cat([x1, x2, x3, x4], dim=1)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [4]:
class RotationCNN(nn.Module):
    def __init__(self, num_classes=11):
        super(RotationCNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(3, 6, 3, 1),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.Conv2d(6, 16, 3, 1, 1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
        )
        self.fc1 = nn.Sequential(
            nn.Linear(57600, 3096),
            nn.ReLU()
        )
        self.fc2 = nn.Linear(3096, num_classes)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x1 = self.layer(x)
        x = torch.rot90(x, k=1, dims=[2, 3])
        x2 = self.layer(x)
        x = torch.rot90(x, k=1, dims=[2, 3])
        x3 = self.layer(x)
        x = torch.rot90(x, k=1, dims=[2, 3])
        x4 = self.layer(x)
        x = torch.cat([x1, x2, x3, x4], 1)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [4]:
class Inception(nn.Module):
    def __init__(self, *dims):
        super(Inception, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(dims[0], dims[1], 1, 1),
            nn.ReLU()
        )
        self.conv1_3 = nn.Sequential(
            nn.Conv2d(dims[0], dims[2], 1, 1),
            nn.ReLU(),
            nn.Conv2d(dims[2], dims[3], 3, 1, 1),
            nn.ReLU(),
        )
        self.conv1_5 = nn.Sequential(
            nn.Conv2d(dims[0], dims[4], 1),
            nn.ReLU(),
            nn.Conv2d(dims[4], dims[5], 5, 1, 2),
            nn.ReLU(),
        )
        self.conv3_1 = nn.Sequential(
            nn.MaxPool2d(3, 1, 1),
            nn.Conv2d(dims[0], dims[6], 1, 1),
            nn.ReLU()
        )

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv1_3(x)
        x3 = self.conv1_5(x)
        x4 = self.conv3_1(x)
        x = torch.cat([x1, x2, x3, x4], 1)
        return x
    
class GoogLeNet(nn.Module):
    def __init__(self, num_classes=11):
        super(GoogLeNet, self).__init__()
        self.num_classes = num_classes
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, 7, 2, 3),
            nn.MaxPool2d(3, 2, 1),
            nn.Conv2d(64, 192, 3, 1, 1),
            nn.MaxPool2d(3, 2, 1)
        )
        self.layer2 = nn.Sequential(
            Inception(192, 64, 96, 128, 16, 32, 32),
            Inception(256, 128, 128, 192, 32, 96, 64),
            nn.MaxPool2d(3, 2, 1),
            Inception(480, 192, 96, 208, 16, 48, 64)
        )
        self.layer3 = nn.Sequential(
            Inception(512, 160, 112, 224, 24, 64, 64),
            Inception(512, 128, 128, 256, 24, 64, 64),
            Inception(512, 112, 144, 288, 32, 64, 64),
        )
        self.layer4 = nn.Sequential(
            Inception(528, 256, 160, 320, 32, 128, 128),
            nn.MaxPool2d(3, 2, 1),
            Inception(832, 256, 160, 320, 32, 128, 128),
            Inception(832, 384, 192, 384, 48, 128, 128),
            nn.AvgPool2d(7, 1)
        )
        self.cf1 = nn.Sequential(
            nn.AvgPool2d(5, 3),
            nn.Conv2d(512, 128, 1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(1024, num_classes)
        )
        self.cf2 = nn.Sequential(
            nn.AvgPool2d(5, 3),
            nn.Conv2d(528, 128, 1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(1024, num_classes)
        )
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x1 = self.cf1(x)
        x = self.layer3(x)
        x2 = self.cf2(x)
        x = self.layer4(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        if self.training:
            return x, x1, x2
        return x

In [13]:
class Inception(nn.Module):
    def __init__(self, *dims):
        super(Inception, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(dims[0], dims[1], 1, 1),
            MoLU()
        )
        self.conv1_3 = nn.Sequential(
            nn.Conv2d(dims[0], dims[2], 1, 1),
            MoLU(),
            nn.Conv2d(dims[2], dims[3], 3, 1, 1),
            MoLU(),
        )
        self.conv1_5 = nn.Sequential(
            nn.Conv2d(dims[0], dims[4], 1),
            MoLU(),
            nn.Conv2d(dims[4], dims[5], 5, 1, 2),
            MoLU(),
        )
        self.conv3_1 = nn.Sequential(
            nn.MaxPool2d(3, 1, 1),
            nn.Conv2d(dims[0], dims[6], 1, 1),
            MoLU()
        )

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv1_3(x)
        x3 = self.conv1_5(x)
        x4 = self.conv3_1(x)
        x = torch.cat([x1, x2, x3, x4], 1)
        return x
    
class MoLeNet(nn.Module):
    def __init__(self, num_classes=11):
        super(MoLeNet, self).__init__()
        self.num_classes = num_classes
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, 7, 2, 3),
            nn.MaxPool2d(3, 2, 1),
            nn.Conv2d(64, 192, 3, 1, 1),
            nn.MaxPool2d(3, 2, 1)
        )
        self.layer2 = nn.Sequential(
            Inception(192, 64, 96, 128, 16, 32, 32),
            Inception(256, 128, 128, 192, 32, 96, 64),
            nn.MaxPool2d(3, 2, 1),
            Inception(480, 192, 96, 208, 16, 48, 64)
        )
        self.layer3 = nn.Sequential(
            Inception(512, 160, 112, 224, 24, 64, 64),
            Inception(512, 128, 128, 256, 24, 64, 64),
            Inception(512, 112, 144, 288, 32, 64, 64),
        )
        self.layer4 = nn.Sequential(
            Inception(528, 256, 160, 320, 32, 128, 128),
            nn.MaxPool2d(3, 2, 1),
            Inception(832, 256, 160, 320, 32, 128, 128),
            Inception(832, 384, 192, 384, 48, 128, 128),
            nn.AvgPool2d(7, 1)
        )
        self.cf1 = nn.Sequential(
            nn.AvgPool2d(5, 3),
            nn.Conv2d(512, 128, 1),
            MoLU(),
            nn.Flatten(),
            nn.Linear(2048, 1024),
            MoLU(),
            nn.Dropout(0.7),
            nn.Linear(1024, num_classes)
        )
        self.cf2 = nn.Sequential(
            nn.AvgPool2d(5, 3),
            nn.Conv2d(528, 128, 1),
            MoLU(),
            nn.Flatten(),
            nn.Linear(2048, 1024),
            MoLU(),
            nn.Dropout(0.7),
            nn.Linear(1024, num_classes)
        )
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x1 = self.cf1(x)
        x = self.layer3(x)
        x2 = self.cf2(x)
        x = self.layer4(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x, x1, x2

In [21]:
class VGG16(nn.Module):
    def __init__(self, num_classes=11):
        super(VGG16, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, 1, 1)
        self.conv2 = nn.Conv2d(64, 64, 3, 1, 1)
        self.conv3 = nn.Conv2d(64, 128, 3, 1, 1)
        self.conv4 = nn.Conv2d(128, 128, 3, 1, 1)
        self.conv5 = nn.Conv2d(128, 256, 3, 1, 1)
        self.conv6 = nn.Conv2d(256, 256, 3, 1, 1)
        self.conv7 = nn.Conv2d(256, 256, 3, 1, 1)
        self.conv8 = nn.Conv2d(256, 512, 3, 1, 1)
        self.conv9 = nn.Conv2d(512, 512, 3, 1, 1)
        self.conv10 = nn.Conv2d(512, 512, 3, 1, 1)
        self.conv11 = nn.Conv2d(512, 512, 3, 1, 1)
        self.conv12 = nn.Conv2d(512, 512, 3, 1, 1)
        self.conv13 = nn.Conv2d(512, 512, 3, 1, 1)
        self.fc14 = nn.Linear(32768, 4096)
        self.fc15 = nn.Linear(4096, 4096)
        self.fc16 = nn.Linear(4096, num_classes)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.pool(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = self.pool(x)
        x = self.conv11(x)
        x = self.conv12(x)
        x = self.conv13(x)
        x = self.pool(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.fc14(x)
        x = self.fc15(x)
        x = self.fc16(x)
        return x

        

In [3]:
class Residual(nn.Module):
    def __init__(self, in_, out_):
        super().__init__()
        self.shortcut1 = nn.Sequential(
            nn.Conv2d(in_, out_, 1),
            nn.BatchNorm2d(out_)
        )
        self.shortcut2 = nn.Sequential(
            nn.Conv2d(out_, out_, 1),
            nn.BatchNorm2d(out_)
        )
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_, out_, 3, 1, 1),
            nn.BatchNorm2d(out_),
            nn.ReLU(),
            nn.Conv2d(out_, out_, 3, 1, 1),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_, out_, 3, 1, 1),
            nn.BatchNorm2d(out_),
            nn.ReLU(),
            nn.Conv2d(out_, out_, 3, 1, 1),
        )
    def forward(self, x):
        x = self.conv1(x) + self.shortcut1(x)
        x = F.relu(x)
        x = self.conv2(x) + self.shortcut2(x)
        x = F.relu(x)
        return x
    
class ResNet(nn.Module):
    def __init__(self, n=11):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, 7, 2, 3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(3, 2, 1)
        )
        self.layer2 = Residual(64, 64)
        self.layer3 = Residual(64, 128)
        self.layer4 = Residual(128, 256)
        self.layer5 = Residual(256, 512)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.clf = nn.Linear(512, n)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.clf(x)
        return x

In [43]:
class ZFNet(nn.Module):
    def __init__(self, num_classes=11):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, 7, 2, 1),
            nn.ReLU(),
            nn.LocalResponseNorm(96),
            nn.MaxPool2d(3, 2, 1)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, 5, 2),
            nn.ReLU(),
            nn.LocalResponseNorm(256),
            nn.MaxPool2d(3, 2, 1)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 512, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(512, 1024, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(1024, 512, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )
        self.layer4 = nn.Sequential(
            nn.Linear(512*6*6, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes)
        )
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.view(x.size(0), -1)
        x = self.layer4(x)
        return x

In [8]:
class LeNet5(nn.Module):
    def __init__(self, num_classes=11):
        super(LeNet5, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            nn.AvgPool2d(2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.AvgPool2d(2),
            nn.Conv2d(16, 120, 5),
        )
        self.fc = nn.Sequential(
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, num_classes)
        )
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
        

In [ ]:
##### OTHERS
class Permute:
    def __call__(x: torch.Tensor):
        x = x.permute(0, 2, 1)
        return x

In [3]:
class VisionTransformer(nn.Module):
    def __init__(self, num_classes=11):
        super().__init__()
        self.cls_token = nn.Parameter(torch.randn(1, 1, 768))
        self.position = nn.Parameter(torch.randn(1, 197, 768))
        self.conv_embed = nn.Conv2d(15, 768, 16, 16)
        self.q = nn.Linear(768, 768)
        self.k = nn.Linear(768, 768)
        self.v = nn.Linear(768, 768)
        self.proj = nn.Linear(768, 768)
        self.ln = nn.LayerNorm(768)
        self.tf_mlp = nn.Sequential(
            nn.LayerNorm(768),
            nn.Linear(768, 768*2),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(768*2, 768),
            nn.Dropout(0.2),
        )
        self.clf = nn.Linear(768, num_classes)
    
    def embedding(self, x):
        x = self.conv_embed(x)
        x = torch.flatten(x, 2, 3)
        x = x.permute(0, 2, 1)  # 1, 196, 768
        x = torch.cat([x, self.cls_token], dim=1)
        x += self.position 
        return x  # 1, 197, 768
    
    def multiHeadAttention(self, x):
        q = self.q(x).view(x.size(0), 8, 197, -1)
        k = self.k(x).view(x.size(0), 8, 197, -1)
        v = self.v(x).view(x.size(0), 8, 197, -1)
        A = torch.matmul(q, k.transpose(2, 3))
        A = F.softmax(A, dim=-1) / (4 * (6**0.5))
        SA = torch.matmul(A, v)
        SA = SA.view(SA.size(0), SA.size(2), -1)
        SA = self.proj(SA)
        return SA
    
    def transformer(self, x):
        res = x
        x = self.ln(x)
        x = self.multiHeadAttention(x) + res
        res = x
        x = self.ln(x)
        x = self.tf_mlp(x) + res
        return x
    
    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        x = torch.mean(x, dim=1)
        x = self.ln(x)
        x = self.clf(x)
        return x

In [32]:
sweep_config = {
    'method': 'bayes'
}

metric = {
    'name': 'valid_acc',
    'goal': 'maximize'
}

parameters_dict = {
    'batch_size': {'values': [4, 8, 16, 32]},
    'learning_rate': {
        'distribution': 'uniform',
        'min': 1e-3,
        'max': 1e-2,
    },
    'optim': {'values': ['Adam', 'SGD', 'Adadelta']}
}

sweep_config['metric'] = metric
sweep_config['parameters'] = parameters_dict

transform = transforms.Compose([
    transforms.Resize((48, 48)),
    #transforms.AutoAugment(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

traindata = CustomDataset('./data/', train=True, transform=transform)
train_len = int(len(traindata)*0.8)
valid_len = len(traindata) - train_len
traindata, validdata = random_split(traindata, [train_len, valid_len])

In [33]:
deivce = 'cpu'
def sweep(config=None):
    with wandb.init(config=config):
        config = wandb.config
        train_loader = DataLoader(traindata, batch_size=config['batch_size'], drop_last=True)
        valid_loader = DataLoader(validdata, batch_size=config['batch_size'], drop_last=True)

        model = CNN().to(device)
        loss_fn = nn.CrossEntropyLoss()
        if config['optim'] == 'Adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
        elif config['optim'] == 'SGD':
            optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9)
        elif config['optim'] == 'Adadelta':
            optimizer = torch.optim.Adadelta(model.parameters(), lr=config['learning_rate'], rho=0.9, eps=1e-8)

        best_validation_acc = 0.0
        for t in range(50):
            print(f"-------------------------- Epoch {t+1} --------------------------")
            train_acc, train_loss = train_loop(model=model,
                                            data_loader=train_loader,
                                            loss_func=loss_fn,
                                            optimizer=optimizer,
                                            device=device)
            valid_acc, valid_loss = evaluate_loop(model=model,
                                                data_loader=valid_loader, 
                                                loss_func=loss_fn,
                                                device=device)
            print(f"Train Accuracy: {100*train_acc:2.5f}%\t|\tTrain Avg Loss: {train_loss}")
            print(f"Valid Accuracy: {100*valid_acc:2.5f}%\t|\tValid Avg Loss: {valid_loss}")
            sample_batch, _ = next(iter(train_loader))
            sample_img = to_pil_image(sample_batch[0])
            sample_img = wandb.Image(
                sample_img,
                caption="sample image"
            )

            images = [to_pil_image(image) for image in sample_batch[:5]]

            wandb.log({
                "train_acc": train_acc,
                "train_loss": train_loss,
                "valid_acc": valid_acc,
                "valid_loss": valid_loss,
                "example": sample_img,
                "examples": [wandb.Image(image) for image in images],
            })

            if valid_acc > best_validation_acc:
                torch.save(model.state_dict(), './cnn.pth')

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="CNN")
wandb.agent(sweep_id, sweep, count=10)

In [4]:
wandb.init(
    # Set the project where this run will be logged
    project="ResNet",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name="resnet18"
)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.RandomPosterize(2, 1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

traindata = CustomDataset('./data/', train=True, transform=transform)
train_len = int(len(traindata)*0.8)
valid_len = len(traindata) - train_len
traindata, validdata = random_split(traindata, [train_len, valid_len])
#traindata += CustomDataset('./data/', train=True, transform=transform2)

device='cpu'

train_loader = DataLoader(traindata, shuffle=True, batch_size=8)
valid_loader = DataLoader(validdata, shuffle=True, batch_size=8)
model = ResNet().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-6)

best_validation_acc = 0.0
for t in range(50):
    print(f"--------------------------- Epoch {t+1} ---------------------------")
    train_acc, train_loss = train_loop(model=model,
                                       data_loader=train_loader,
                                       loss_func=loss_fn,
                                       optimizer=optimizer,
                                       transform=transforms.RandomPerspective(0.1, 0.01),
                                       device=device)
    valid_acc, valid_loss = evaluate_loop(model=model,
                                          data_loader=valid_loader, 
                                          loss_func=loss_fn,
                                          device=device)
    print(f"Train Accuracy: {100*train_acc:2.5f}%\t|\tTrain Avg Loss: {train_loss:2.5f}")
    print(f"Valid Accuracy: {100*valid_acc:2.5f}%\t|\tValid Avg Loss: {valid_loss:2.5f}")
    sample_batch, _ = next(iter(train_loader)) 
    #sample_img = to_pil_image(sample_batch[0])
    sample_img = to_pil_image(sample_batch[0, :3, :, :])
    sample_img = wandb.Image( # single image
        sample_img,
        caption="sample image"
    )

    wandb.log({
        "train_acc": train_acc,
        "valid_acc": valid_acc,
        "train_loss": train_loss,
        "valid_loss": valid_loss,
        "example": sample_img,
    })

    if valid_acc > best_validation_acc:
        torch.save(model.state_dict(), './resnet.pth')

--------------------------- Epoch 1 ---------------------------


100%|██████████| 20/20 [00:28<00:00,  1.40s/it]


Train Accuracy: 8.21256%	|	Train Avg Loss: 2.45734
Valid Accuracy: 2.56410%	|	Valid Avg Loss: 2.40249
--------------------------- Epoch 2 ---------------------------


  0%|          | 0/78 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [7]:
wandb.init(
    # Set the project where this run will be logged
    project="ViT",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name="VisionTransformer"
)

transform = transforms.Compose([
    transforms.Resize((240, 240)),
    #transforms.CenterCrop(224),
    #transforms.RandomPosterize(4, 1),
    transforms.ToTensor(),
    transforms.RandomAdjustSharpness(2),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.FiveCrop(224),
    transforms.Lambda(lambda x: torch.cat(x)),
])

traindata = CustomDataset('./data/', train=True, transform=transform)
train_len = int(len(traindata)*0.8)
valid_len = len(traindata) - train_len
traindata, validdata = random_split(traindata, [train_len, valid_len])

device='cpu'

train_loader = DataLoader(traindata, shuffle=True, batch_size=1)
valid_loader = DataLoader(validdata, shuffle=True, batch_size=1)
model = VisionTransformer()#.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-6)

best_validation_acc = 0.0
for t in range(100):
    print(f"--------------------------- Epoch {t+1} ---------------------------")
    train_acc, train_loss = train_loop(model=model,
                                       data_loader=train_loader,
                                       loss_func=loss_fn,
                                       optimizer=optimizer,
                                       device=device)
    valid_acc, valid_loss = evaluate_loop(model=model,
                                          data_loader=valid_loader, 
                                          loss_func=loss_fn,
                                          device=device)
    print(f"Train Accuracy: {100*train_acc:2.5f}%\t|\tTrain Avg Loss: {train_loss:2.5f}")
    print(f"Valid Accuracy: {100*valid_acc:2.5f}%\t|\tValid Avg Loss: {valid_loss:2.5f}")
    sample_batch, _ = next(iter(train_loader)) 
    sample_img = to_pil_image(sample_batch[0, :3, :, :])
    sample_img = wandb.Image( # single image
        sample_img,
        caption="sample image"
    )

    wandb.log({
        "train_acc": train_acc,
        "valid_acc": valid_acc,
        "train_loss": train_loss,
        "valid_loss": valid_loss,
        "example": sample_img,
    })

    if valid_acc > best_validation_acc:
        torch.save(model.state_dict(), './vit.pth')

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


train_acc,▁
train_loss,▁
valid_acc,▁
valid_loss,▁
train_acc,0.11594
train_loss,2.82976
valid_acc,0.13462
valid_loss,2.60938


--------------------------- Epoch 1 ---------------------------


100%|██████████| 156/156 [00:14<00:00, 10.89it/s]


Train Accuracy: 9.33977%	|	Train Avg Loss: 2.92411
Valid Accuracy: 6.41026%	|	Valid Avg Loss: 2.51015
--------------------------- Epoch 2 ---------------------------


...

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


train_acc,▁▁▂▂▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇██▇█
train_loss,█▆▆▆▅▅▅▄▅▄▄▄▃▃▃▃▃▃▃▂▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁
valid_acc,▁▂▃▃▃▄▄▄▄▅▄▅▆▅▅▅▆▆▆▆▇▇▇▅▆▇▇▆▇▇▆▆▆▇▇▇████
valid_loss,██▆▇▆▅▄▄▄▃▄▄▃▃▃▄▂▂▂▃▂▂▂▃▂▁▂▁▁▂▃▄▄▁▃▂▁▂▁▁
train_acc,0.73913
train_loss,0.69598
valid_acc,0.58974
valid_loss,1.5265


--------------------------- Epoch 1 ---------------------------


  0%|          | 0/39 [00:01<?, ?it/s]


TypeError: Variable data has to be a tensor, but got list

In [7]:
wandb.init(
    # Set the project where this run will be logged
    project="GoogLeNet",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name="googlenet"
)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

traindata = CustomDataset('./data/', train=True, transform=transform)
train_len = int(len(traindata)*0.8)
valid_len = len(traindata) - train_len
traindata, validdata = random_split(traindata, [train_len, valid_len])

device='cpu'

train_loader = DataLoader(traindata, shuffle=True, batch_size=16)
valid_loader = DataLoader(validdata, shuffle=True, batch_size=16)
model = GoogLeNet()#.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002, weight_decay=1e-6)

best_validation_acc = 0.0
for t in range(100):
    print(f"--------------------------- Epoch {t+1} ---------------------------")
    train_acc, train_loss = train_loop(model=model,
                                       data_loader=train_loader,
                                       loss_func=loss_fn,
                                       optimizer=optimizer,
                                       transform=transforms.RandomPerspective(0.2),
                                       google=True,
                                       device=device)
    valid_acc, valid_loss = evaluate_loop(model=model,
                                          data_loader=valid_loader, 
                                          loss_func=loss_fn,
                                          device=device)
    print(f"Train Accuracy: {100*train_acc:2.5f}%\t|\tTrain Avg Loss: {train_loss:2.5f}")
    print(f"Valid Accuracy: {100*valid_acc:2.5f}%\t|\tValid Avg Loss: {valid_loss:2.5f}")
    sample_batch, _ = next(iter(train_loader)) 
    sample_img = to_pil_image(sample_batch[0, :3, :, :])
    sample_img = wandb.Image( # single image
        sample_img,
        caption="sample image"
    )

    wandb.log({
        "train_acc": train_acc,
        "valid_acc": valid_acc,
        "train_loss": train_loss,
        "valid_loss": valid_loss,
        "example": sample_img,
    })

    if valid_acc > best_validation_acc:
        best_validation_acc = valid_acc
        torch.save(model.state_dict(), './googlenet.pth')

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


train_acc,█▁▃
train_loss,█▇▁
valid_acc,▁▁▁
valid_loss,▁▂█
train_acc,0.08052
train_loss,60.8919
valid_acc,0.0641
valid_loss,37.45723


--------------------------- Epoch 1 ---------------------------


100%|██████████| 10/10 [00:20<00:00,  2.06s/it]


Train Accuracy: 9.17874%	|	Train Avg Loss: 3.83898
Valid Accuracy: 4.48718%	|	Valid Avg Loss: 2.40074
--------------------------- Epoch 2 ---------------------------


...

Train Accuracy: 96.13527%	|	Train Avg Loss: 0.19546
Valid Accuracy: 80.12821%	|	Valid Avg Loss: 0.80800
--------------------------- Epoch 99 ---------------------------


100%|██████████| 10/10 [00:22<00:00,  2.24s/it]


Train Accuracy: 96.94042%	|	Train Avg Loss: 0.14384
Valid Accuracy: 82.69231%	|	Valid Avg Loss: 0.76771
--------------------------- Epoch 100 ---------------------------


100%|██████████| 10/10 [00:21<00:00,  2.12s/it]


Train Accuracy: 98.06763%	|	Train Avg Loss: 0.13637
Valid Accuracy: 82.05128%	|	Valid Avg Loss: 0.85213


In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
model_origin = model
#model_.load_state_dict(torch.load('./googlenet.pth'))

valid_acc, valid_loss = evaluate_loop(model=model_origin,
                                      data_loader=valid_loader, 
                                      loss_func=loss_fn,
                                      device=device)
print(f"FINAL : Valid Accuracy: {100*valid_acc:2.5f}%\t|\tValid Avg Loss: {valid_loss:2.5f}")

model_best = model
model_best.load_state_dict(torch.load('./googlenet.pth'))
valid_acc, valid_loss = evaluate_loop(model=model_best,
                                      data_loader=valid_loader, 
                                      loss_func=loss_fn,
                                      device=device)
print(f"BEST : Valid Accuracy: {100*valid_acc:2.5f}%\t|\tValid Avg Loss: {valid_loss:2.5f}")



100%|██████████| 10/10 [00:20<00:00,  2.07s/it]


FINAL : Valid Accuracy: 82.05128%	|	Valid Avg Loss: 0.85261


100%|██████████| 10/10 [00:20<00:00,  2.07s/it]

BEST : Valid Accuracy: 82.69231%	|	Valid Avg Loss: 0.84879


In [ ]:
testdata = CustomDataset('./data/', train=False, transform=transform)
test_loader = DataLoader(testdata)

test_predict(model=model_best,
             test_loader=test_loader,
             path='./',
             file_name='googlenet_.csv',
             device='cpu')

In [ ]:
wandb.init(
    # Set the project where this run will be logged
    project="GoogLeNet",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name="googlenet"
)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomPosterize(3, 1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

traindata = CustomDataset('./data/', train=True, transform=transform)
train_len = int(len(traindata)*0.8)
valid_len = len(traindata) - train_len
traindata, validdata = random_split(traindata, [train_len, valid_len])
#traindata += CustomDataset('./data/', train=True, transform=transform2)

device='cpu'

train_loader = DataLoader(traindata, shuffle=True, batch_size=16)
valid_loader = DataLoader(validdata, shuffle=True, batch_size=16)
model = GoogLeNet()#.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.NAdam(model.parameters(), lr=0.0005, weight_decay=1e-6)

best_validation_acc = 0.0
for t in range(100):
    print(f"--------------------------- Epoch {t+1} ---------------------------")
    train_acc, train_loss = train_loop(model=model,
                                       data_loader=train_loader,
                                       loss_func=loss_fn,
                                       optimizer=optimizer,
                                       google=True,
                                       transform=transforms.RandomPerspective(0.1, 0.01),
                                       device=device)
    valid_acc, valid_loss = evaluate_loop(model=model,
                                          data_loader=valid_loader, 
                                          google=True,
                                          loss_func=loss_fn,
                                          device=device)
    print(f"Train Accuracy: {100*train_acc:2.5f}%\t|\tTrain Avg Loss: {train_loss:2.5f}")
    print(f"Valid Accuracy: {100*valid_acc:2.5f}%\t|\tValid Avg Loss: {valid_loss:2.5f}")
    sample_batch, _ = next(iter(train_loader)) 
    #sample_img = to_pil_image(sample_batch[0])
    sample_img = to_pil_image(sample_batch[0, :3, :, :])
    sample_img = wandb.Image( # single image
        sample_img,
        caption="sample image"
    )

    wandb.log({
        "train_acc": train_acc,
        "valid_acc": valid_acc,
        "train_loss": train_loss,
        "valid_loss": valid_loss,
        "example": sample_img,
    })

    if valid_acc > best_validation_acc:
        torch.save(model.state_dict(), './googlenet.pth')

In [6]:
transform = transforms.Compose([
    transforms.Resize((240, 240)),
    #transforms.CenterCrop(224),
    #transforms.RandomPosterize(4, 1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.FiveCrop(224),
    transforms.Lambda(lambda x: torch.cat(x)),
])

testset = CustomDataset('./data/', train=False, transform=transform)
test_loader = DataLoader(testset)

model_ = VisionTransformer()
model_.load_state_dict(torch.load('./vit.pth'))
test_predict(model=model_,
             test_loader=test_loader,
             path='./',
             file_name='vit.csv',
             device='cpu')

100%|██████████| 330/330 [00:34<00:00,  9.50it/s]

Model Prediction was saved at ./vit.csv
